# 준비

먼저 Naver Cloud Platform에서 지오코딩 API를 사용할 수 있도록 API Key를 받습니다.  
https://api.ncloud-docs.com/docs/ko/ai-naver-mapsgeocoding 

In [1]:
# 데이터 처리
import numpy as np
import pandas as pd

In [2]:
# "전국CCTV표준데이터" CSV 파일 로딩 
df_cctv = pd.read_csv('./data/전국CCTV표준데이터.csv', encoding="EUC-KR", header=0, 
                           names=['admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 'pixel', 'direction', 'period', 'date_install', 'tel', 'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'], 
                           dtype={'cctv_cnt':object, 'pixel':object, 'period':object, 'lat':float, 'lon':float, 'offer_admin_cd':object}, 
                  thousands = ',', #천단위 쉼표 제거   
                  index_col=None) 

In [4]:
df_cctv.dropna(subset=['lat'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['lon'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['addr_rd'],axis=0,inplace=True) # 도로명주소 nulll 데이터 삭제
df_cctv.dropna(subset=['addr_jb'],axis=0,inplace=True) #지번주소 null 데이터 삭제 

In [5]:
# 5만 건 중 0.01% 샘플링
df_cctv2 = df_cctv.sample(frac=0.001)
df_cctv2.info()
df_cctv2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 30327 to 29145
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   admin_nm        24 non-null     object 
 1   addr_rd         24 non-null     object 
 2   addr_jb         24 non-null     object 
 3   purpose         24 non-null     object 
 4   cctv_cnt        24 non-null     object 
 5   pixel           23 non-null     object 
 6   direction       22 non-null     object 
 7   period          24 non-null     object 
 8   date_install    21 non-null     object 
 9   tel             24 non-null     object 
 10  lat             24 non-null     float64
 11  lon             24 non-null     float64
 12  date_make       24 non-null     object 
 13  offer_admin_cd  24 non-null     object 
 14  offer_admin_nm  24 non-null     object 
dtypes: float64(2), object(13)
memory usage: 3.0+ KB


,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
30327,대구광역시 서구청,대구광역시 서구 새방로25길 46,대구광역시 서구 상리동 644,다목적,1,200,남,30,2011-12,053-663-3901,35.866413,128.525622,2020-12-24,3430000,대구광역시 서구
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,37.329761,127.944557,2020-09-20,4190000,강원도 원주시
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,36.462876,127.127437,2020-03-25,4500000,충청남도 공주시
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,35.225864,129.093181,2020-08-12,3350000,부산광역시 금정구
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,35.242717,128.680032,2020-08-25,5670000,경상남도 창원시


In [13]:
import urllib
import json

In [14]:
# 네이버 API 정보 읽기 
f = open('../../etc/naver.txt', 'r')
lines = f.readlines()

api_id = lines[0].replace("\n", "")
api_key = lines[1].replace("\n", "")

In [15]:
# 네이버 Map API https://console.ncloud.com/mc/solution/naverService/application
def search_map(search_text):
    client_id = 'id'
    client_secret = 'secret'
    encText = urllib.parse.quote(str(search_text))
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='+encText
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', api_id) 
    request.add_header('X-NCP-APIGW-API-KEY', api_key) 
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body.decode('utf-8'))
        return response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

In [16]:
# 단 건 테스트
x = []
y = []

temp_map = search_map('경기 부천시  소사본1동 181-26 2층 201호')  #NAVER search_map 함수 실행
temp_map = json.loads(temp_map)
try:
    #print(temp_map)
    temp_map = temp_map['addresses'][0]
    x = temp_map['x']
    y = temp_map['y']
except IndexError:
    pass

print(x, y)

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"","jibunAddress":"경기도 부천시 소사본동 181-26","englishAddress":"181-26, Sosabon-dong, Bucheon-si, Gyeonggi-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"경기도","shortName":"경기도","code":""},{"types":["SIGUGUN"],"longName":"부천시","shortName":"부천시","code":""},{"types":["DONGMYUN"],"longName":"소사본동","shortName":"소사본동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NUMBER"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"181-26","shortName":"181-26","code":""},{"types":["POSTAL_CODE"],"longName":"","shortName":"","code":""}],"x":"126.7953156","y":"37.4748725","distance":0.0}],"errorMessage":""}
126.7953156 37.4748725


In [17]:
x1 = []
y1 = []
length_df = len(df_cctv2)
print(length_df)

15


In [18]:
# 데이터프레임 행수만큼 읽으면서 addr 항목의 주소를 Naver API로 돌려서 좌표값을 받아 리스트에 추가  
for i in range(length_df):
    addr = df_cctv2.iloc[i]['addr_rd']
    #print(addr)
    #print(i)
    temp_map = search_map(addr)  #NAVER search_map 함수 실행
    temp_map = json.loads(temp_map)
    try:
        #print(temp_map)
        temp_map = temp_map['addresses'][0]
        x = temp_map['x']
        y = temp_map['y']
        x1.append(x)
        y1.append(y)    
    except:
        x1.append(0)
        y1.append(0)    

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"강원도 원주시 소방서길 22-5","jibunAddress":"강원도 원주시 명륜동 820-6","englishAddress":"22-5, Sobangseo-gil, Wonju-si, Gangwon-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"강원도","shortName":"강원도","code":""},{"types":["SIGUGUN"],"longName":"원주시","shortName":"원주시","code":""},{"types":["DONGMYUN"],"longName":"명륜동","shortName":"명륜동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"소방서길","shortName":"소방서길","code":""},{"types":["BUILDING_NUMBER"],"longName":"22-5","shortName":"22-5","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"820-6","shortName":"820-6","code":""},{"types":["POSTAL_CODE"],"longName":"26481","shortName":"26481","code":""}],"x":"127.9444656","y":"37.3297418","distance":0.0}],"errorMessage":""}
{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"경상남도 하동군 화개면 쌍계로 15 화개장터","jibunAddress":"경상남도 하동군 화개면 탑리 726-46 화개장터","englishAddress":"15, Ssanggye-ro, Hwagae-myeon, Hadong-gun, Gyeongsangnam-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"경상남도","shortName":"경상남도","code":""},{"types":["SIGUGUN"],"longName":"하동군","shortName":"하동군","code":""},{"types":["DONGMYUN"],"longName":"화개면","shortName":"화개면","code":""},{"types":["RI"],"longName":"탑리","shortName":"탑리","code":""},{"types":["ROAD_NAME"],"longName":"쌍계로","shortName":"쌍계로","code":""},{"types":["BUILDING_NUMBER"],"longName":"15","shortName":"15","code":""},{"types":["BUILDING_NAME"],"longName":"화개장터","shortName":"화개장터","code":""},{"types":["LAND_NUMBER"],"longName":"726-46","shortName":"726-46","code":""},{"types":["POSTAL_CODE"],"longName":"52303","shortName":"52303","code":""}],"x":"127.6240669","y":"35.1879895","distance":0.0}],"errorMessage":""}
{"status":"OK","meta":{

In [19]:
# 좌표 리스트 값을 데이터프레임의 X, Y 항목에 넣어줌 
df_cctv2['x'] = x1
df_cctv2['y'] = y1

In [20]:
# 잘못 입력된 좌표 리턴값 "[]"을 치환
df_cctv2['x'] = df_cctv2['x'].str.replace('[', '0')
df_cctv2['x'] = df_cctv2['x'].str.replace(']', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace('[', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace(']', '0')
df_cctv2.head()

<ipython-input-20-30e685e7b4a0>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['x'] = df_cctv2['x'].str.replace('[', '0')
<ipython-input-20-30e685e7b4a0>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['x'] = df_cctv2['x'].str.replace(']', '0')
<ipython-input-20-30e685e7b4a0>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['y'] = df_cctv2['y'].str.replace('[', '0')
<ipython-input-20-30e685e7b4a0>:5: FutureWarning: The default value of regex will change from True to False in a future version. I

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,...,lon,date_make,offer_admin_cd,offer_admin_nm,location,lat2,lon2,dist,x,y
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,...,127.944557,2020-09-20,4190000,강원도 원주시,"(소방서길, 명륜동, 원주시, 강원도, 26481, 대한민국, (37.3316091...",37.331609,127.944570,205.447154,127.9444656,37.3297418
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,...,127.127437,2020-03-25,4500000,충청남도 공주시,"(금벽로, 노은1동, 노은동, 온천리, 공주시, 충청남도, 34158, 대한민국, ...",36.363186,127.268690,16812.226790,127.1274390,36.4702150
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,...,129.093181,2020-08-12,3350000,부산광역시 금정구,"(동부곡로15번길, 부곡2동, 부곡동, 금정구, 부산, 46271, 대한민국, (3...",35.228317,129.093780,278.154903,129.0931920,35.2258501
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,...,128.680032,2020-08-25,5670000,경상남도 창원시,"(창이대로397번길, 의창구, 창원시, 경상남도, 51431, 대한민국, (35.2...",35.244254,128.680770,183.549756,128.6802964,35.2430056
31971,전라북도 군산시,전라북도 군산시 번영로 2,전라북도 군산시 경장동 472,생활방범,2,200,360도 전방면,30,2017-12,063-454-7922,...,126.723794,2020-03-12,4670000,전라북도 군산시,"(번영로, 대야면, 군산시, 전라북도, 54060, 대한민국, (35.9413032...",35.941303,126.831394,10391.407279,126.7238333,35.9751429


In [21]:
# 좌표 항목을 실수 형식으로 변환 
df_cctv2['x'] = df_cctv2['x'].astype(float)
df_cctv2['y'] = df_cctv2['y'].astype(float)

In [72]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.706381,35.974254
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.088554,37.623989
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.374198,36.407927
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.428529,35.504509
7419,서울특별시 마포구청,와우산로30길,창전동 3-183,방범(공원),1,200,360도전방면,30,2012-07,02-3153-8432,37.552997,126.930324,2020-05-20,3130000,서울특별시 마포구,NaN,NaN
25390,부산광역시 수영구청,부산광역시 수영구 장대골로27번길 29,부산광역시 수영구 광안동 735-11,생활방범,1,200,360도전방면,30,2015-11,051-610-4492,35.157912,129.110367,2019-12-20,3380000,부산광역시 수영구,129.110358,35.157911
41489,전라북도 전주시청,전라북도 전주시 완산구 안행2길 25,전라북도 전주시 완산구 삼천동1가 739,어린이보호,2,200,반경100m이내,30,2012-12,063-281-2072,35.802009,127.129149,2020-08-31,4640000,전라북도 전주시,127.128724,35.801831
11684,서울특별시 강북구청,서울특별시 강북구 삼양로139가길 43,서울특별시 강북구 우이동 73-23,생활방범,1,200,NaN,30,2012-05,02-901-7266,37.651230,127.012093,2020-05-12,3080000,서울특별시 강북구,127.012015,37.651108
24365,안전관리과,경상북도 칠곡군 기산면 행정리 1180-1,경상북도 칠곡군 기산면 행정리 1180-1,차량방범,1,200,NaN,30,2014-07,054-979-6881,35.969625,128.363002,2018-08-22,5220000,경상북도 칠곡군,128.363010,35.969626
29014,서울특별시 성동구청,서울특별시 성동구 자동차시장길 22,서울특별시 성동구 용답동 223-5,생활방범,3,200,360도전방면,30,2014-09,02-2286-5879,37.561981,127.057297,2020-12-04,3030000,서울특별시 성동구,127.056381,37.561185


In [22]:
# 좌표가 null인 데이터 삭제 
df_cctv2.dropna(subset=['x'],axis=0,inplace=True)  
df_cctv2.dropna(subset=['y'],axis=0,inplace=True)  

포털지도와 구글맵 등을 이용하여 도로명주소, 지번주소, 원 좌표, 도로명주소 좌표 등을 비교

In [81]:
#다음 과정을 위해 데이터프레임을 csv 파일로 저장 
df_cctv2.to_csv("./data/cctv_coord_sample.csv", encoding='euc-kr')